In [2]:
# import miscellaneous modules
import matplotlib.pyplot as plt
import os
import numpy as np
import time
import keras
import tensorflow as tf
import tensorflow_datasets as tfds
keras.__version__
tf.__version__

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

2024-10-03 11:14:41.540319: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 11:14:41.551136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 11:14:41.563617: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 11:14:41.567003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 11:14:41.575633: I tensorflow/core/platform/cpu_feature_guar

1 Physical GPUs, 1 Logical GPU


I0000 00:00:1727928883.848477   38650 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928883.894482   38650 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928883.894532   38650 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928883.897800   38650 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727928883.897835   38650 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
(train_ds, test_ds), info = tfds.load("fashion_MNIST", 
                                      split=['train[:5%]', 'test[:5%]'],
                                      as_supervised=True, 
                                      with_info=True)
# Load the Fashion MNIST dataset
# (train_ds, test_ds), info = tfds.load('fashion_mnist', split=['train', 'test'], with_info=True, as_supervised=True)


In [5]:
#info
import numpy as np

def dataset_to_numpy(dataset):
    images = []
    labels = []
    for image, label in tfds.as_numpy(dataset):
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

# Convert train and test datasets to NumPy arrays
train_images, train_labels = dataset_to_numpy(train_ds)
test_images, test_labels = dataset_to_numpy(test_ds)


2024-10-03 11:15:18.246715: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-10-03 11:15:18.366840: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
train_images.shape

(3000, 28, 28, 1)

ข้อเสียหลักๆของ datagen คือจะรับแค่รูปที่เป็น ```numpy.ndarray``` เท่านั้น ซึ่งอาจจะลำบากถ้าเราไปโหลด dataset มาจาก tensorflow ส่วนใหญ่แล้วจะโหลดมาแล้วจะอยู่ในรูปแบบของ tfds.datasets ที่ต้องอาศัยความชำนาญในการใช้งานพอตัว

นั่นเป็นเหตุผลว่าทำไมต้องทำการแปลง dataset ที่โหลดมาให้อยู่ในรูปแบบของ Numpy

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest'
)

datagen.fit(train_images)

ต่อมาต้องเรียกฟังก์ชัน flow โดยการเรียกฟังก์ชันนี้เป็นเหมือนการนำรูปจากใน ```train_images``` ไป flow ผ่าน ```datagen``` ที่เราได้ประกาศไว้ข้างต้น
```train_images```  จะไม่ถูกเพิ่มขนาดแต่อย่างได


ฟังก์ชันนี้มีข้อดีอยู่ว่า เราสามารถเซฟรูปที่ผ่าน Augmentation ออกมาได้ด้วยนั่นเอง ผ่านพารามิเตอร์ ```save_to_dir=``` ที่จะบอก directory ส่วน ```save_prefix=``` บอกว่าให้ขื้นต้นชื่อไฟล์ว่าอะไร 

In [8]:
aug_datagen = datagen.flow(train_images, train_labels, save_to_dir="runs",  save_prefix='aug_')
for _ in range(1):
    img,label = next(aug_datagen)

In [9]:
model = tf.keras.Sequential([
  # Add the preprocessing layers you created earlier.
  tf.keras.Input(shape=(28,28,1)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(10,activation="softmax")
])

In [10]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy']
             )
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,578 (787.41 KB)

 Trainable params: 201,578 (787.41 KB)

 Non-trainable params: 0 (0.00 B)

จากที่อธิบายไว้ก่อนหน้านี้ว่า ฟังก์ชัน flow เป็นการผ่านรูปเข้าไปใน datagen ของเราทำให้เกิดภาพใหม่ๆขึ้นมา(augmented)
แต่นั่นเป็นการเรียกเพียงครั้งเดียวเท่านั้น ซึ่งตากจากโค้ดด้านล่างอยู่

โค้ดด้านล่างเป็นการเรียก ```flow()``` แทนการใส่ ```train_images``` ตรงๆ ซึ่งการทำแบบนี้ทำให้เวลาโมเดลเราเทรนจบและกำลังจะขึ้นรอบใหม่ คำสั่ง ```flow()``` จะถูกเรียกใช้งานอีกรอบ หมายความว่าอย่างไร? หมายความว่าแต่ละ epoch รูปจะถูก augment ขึ้นมาใหม่ทุกๆรอบทำให้ แล้วถ้านับว่าหมุน 1 องศาคือรูปใหม่แล้วหละก็ จำนวนรูปที่ไม่เหมือนกันเลยทั้งหมดจะมีค่าเท่ากับ ```len(train_images) * epoch```

In [26]:
model.fit(datagen.flow(train_images, train_labels), epochs=10)

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8836 - loss: 0.3396
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8650 - loss: 0.3832
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8912 - loss: 0.3198
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8844 - loss: 0.3166
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8813 - loss: 0.3254
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8815 - loss: 0.3166
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8816 - loss: 0.3178
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8744 - loss: 0.3307
Epoch 9/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8981 - loss: 0.2883
Epoch 10/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9038 - loss: 0.3014
